In [2]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.linear_model import LinearRegression, LassoCV
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2
import itertools
import sys
sys.path.append('/home/roquero/CausalAggregation/Code')
from generateEnvironment import GenerateEnvironment, generate_constraints
from gmm_estimator import SolveProblem
import matplotlib as mpl
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
np.set_printoptions(suppress=True)
np.set_printoptions(precision=5)

In [3]:
n_dim=200
connectivity_e0 = np.zeros((n_dim,n_dim))
connectivity_e0[np.arange(1,n_dim), np.arange(0,n_dim-1)] = 1
connectivity_e0[1:,0]=0
y_index=30
x_indices=np.hstack([np.arange(1,y_index),(np.arange(y_index+1,n_dim))])
n_reps=100
alpha=0.05

In [8]:
for n_samples in np.array([50,100,200,500,1000,5000,10000]):
    coverage=[]
    length=[]
    for _ in np.arange(n_reps):
        
        solver_obs = SolveProblem(connectivity_e0,x_indices,y_index)
        obs_dataset = solver_obs.generate_intervention(n_samples, {})['dataset']
    
        select_markov = LassoCV(max_iter=5000)
        _ = select_markov.fit(obs_dataset[x_indices,:].T,obs_dataset[y_index,:])
        selected_indices = x_indices[np.where(np.abs(select_markov.coef_)>1e-3)[0]]
        
        solver = SolveProblem(connectivity_e0,selected_indices,y_index)
        
        list_dict_interventions = [{i:{'type':'independent'} for i in selected_indices}]       
        list_environments = [solver.generate_intervention(n_samples, dict_interventions)
                             for dict_interventions in list_dict_interventions]
        
        beta_hat = solver.compute_beta(list_environments)
        aCov = solver.compute_aCov(list_environments)
        
        CI = solver.compute_CI(beta_hat, aCov, n_samples_tot, alpha)
        
        for coord in np.arange(5):
            coverage.append((CI[0,coord]<solver.beta[coord])&
                        (CI[1,coord]>solver.beta[coord]))
            length.append(CI[1,coord]-CI[0,coord])
            
    print('For n_samples {}: Avg coverage: {} pm{}. Avg length: {} pm{}.'.format(n_samples,np.mean(coverage),2*np.std(coverage)/np.sqrt(n_reps),np.mean(length),2*np.std(length)/np.sqrt(n_reps))
         )

NameError: name 'n_samples_tot' is not defined

In [ ]:
solver.beta